In [37]:
import pandas as pd

In [38]:
orders = pd.read_excel('orders-test.xlsx')
graphic = pd.read_excel('graphic-test.xlsx')

In [39]:
orders

,КССС мат,Пункт отгрузки,"осталось произвести, тн"
0,1779443,ВНП,8


In [40]:
graphic

,Пункт отгрузки,Код КССС,Дата_производства,"Объем_фасовки, т"
0,ВНП,1779443,2022-06-11,8


In [41]:
def data_shipment(city, code, required_to_make):
    print('НАЧАЛО:')
    print(graphic)
    print('required_to_make: ', required_to_make)
    ## проверяем есть ли вообще в списке graphic
    if graphic[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code)].empty:
        return '-'
    ## если в таблице orders не требуется, то мы ставим черточку
    elif required_to_make == 0:
        return '-'
    ## иначе если есть и объема в графике достаточно, то мы    
    elif graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code), 'Объем_фасовки, т'].iloc[0] >= required_to_make:   
        print('Объем больше')
        
        ## вычитаем из объема
        ## добавить определенное дату, так как будет удалять все значения где код и пункт совпадаем
        data_of_making = graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code) & 
                                    (graphic['Дата_производства']==graphic.loc[(graphic['Пункт отгрузки']==city) 
                                    & (graphic['Код КССС']==code), 'Дата_производства'].iloc[0]),'Дата_производства'].iloc[0]
        graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code) & (graphic['Дата_производства']==data_of_making),
                    'Объем_фасовки, т'] -= required_to_make
        # и возвращаем дату
        data = graphic[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code)]['Дата_производства'].dt.date.iloc[0]
        print('graphic1: ', graphic)
        return data
    ## иначе если недостаточно объема чем требуется ...
    else:
        print('объем меньше')
        ## иначе если возвращается только одна дата, возвращаем -
        if len(graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code)]) == 1:
            #graphic.drop(graphic[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code)].index, inplace=True)
            print('graphic3', graphic)
        #return graphic[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code)]['Дата_производства'].dt.date.iloc[0] 
            return "-"
        
        
        ## и возращается несколько дат
        elif len(graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code)]) > 1:
            ## вводим переменную длины, чтобы не выйти за пределы
            len_of_dates = len(graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code)])
            ## вводим счетчик для того чтобы присвоить правильную дату
            i = 0
            ## создаем накопительную переменную 
            storage_var = graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code), 'Объем_фасовки, т'].iloc[0]
            list_of_dates_to_be_deleted = []

            ## пока объем фасовки меньше, чем требуется
            while storage_var<required_to_make and len_of_dates>1:
                print('в цикле')
                print('len_of_dates=', len_of_dates)
                len_of_dates-=1
                ## переходим к следующей
                i+=1
                ## и прибавляем к накопительной переменной
                storage_var+= graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code), 'Объем_фасовки, т'].iloc[i]
                print('прибавили к накопительной следующий объем: ', storage_var)
                list_of_dates_to_be_deleted.append(graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code) & 
                                             (graphic['Дата_производства']==graphic.loc[(graphic['Пункт отгрузки']==city) 
                                            & (graphic['Код КССС']==code), 'Дата_производства'].iloc[i-1]),
                                             'Дата_производства'].iloc[0])
                print('даты которые нужно удалить: ', list_of_dates_to_be_deleted)
                 ## и обнуляем в графике объем фасовки
                #graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code), 'Объем_фасовки, т'].iloc[i]
                
                print('storage fresh, ', storage_var)
                print('required fresh, ' , required_to_make) 
                
            #если требуемый объем не накопился то возвращает -
            if storage_var<required_to_make:
                print('не накопился')
                return '-'
            # иначе
            else:
                ##нужно обнулить или удалить предыдущие даты, которые схлопнулись
                for j in (list_of_dates_to_be_deleted):
                    graphic.drop(graphic[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code) & 
                           (graphic['Дата_производства']==j)].index, inplace=True)
                    i-=1
                print('i= ',i)
                
                ## предыдущая дата и текущая
#                 data_of_making_pred = graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code) & 
#                                              (graphic['Дата_производства']==graphic.loc[(graphic['Пункт отгрузки']==city) 
#                                             & (graphic['Код КССС']==code), 'Дата_производства'].iloc[i-1]),
#                                              'Дата_производства'].iloc[0]
                data_of_making = graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code) & 
                                             (graphic['Дата_производства']==graphic.loc[(graphic['Пункт отгрузки']==city) 
                                            & (graphic['Код КССС']==code), 'Дата_производства'].iloc[i]),
                                             'Дата_производства'].iloc[0]
                
                ##попробуем не обнулять, а сразу удалять график которого объем стал равен нулю
#                 graphic.drop(graphic[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code) & 
#                             (graphic['Дата_производства']==data_of_making_pred)].index, inplace=True)
                print('после удаления:, ',graphic)
                

            print('data making', data_of_making)
        
            ## вычитаем
            graphic.loc[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code) & 
                        (graphic['Дата_производства']==data_of_making), 'Объем_фасовки, т'] = storage_var-required_to_make
            print('вычли')


            print('по окончанию цикла возвращает тот i который требуется', i)    
            data = graphic[(graphic['Пункт отгрузки']==city) & (graphic['Код КССС']==code)]['Дата_производства'].dt.date.iloc[i] 
            print('graphic2', graphic)
            return data

        
        
        


In [42]:
orders['дата производства'] = orders.apply(lambda x: data_shipment(x['Пункт отгрузки'], x['КССС мат'], x['осталось произвести, тн']), axis=1)

НАЧАЛО:
  Пункт отгрузки  Код КССС Дата_производства  Объем_фасовки, т
0            ВНП   1779443        2022-06-11                 8
required_to_make:  8
Объем больше
graphic1:    Пункт отгрузки  Код КССС Дата_производства  Объем_фасовки, т
0            ВНП   1779443        2022-06-11                 0


In [43]:
graphic

,Пункт отгрузки,Код КССС,Дата_производства,"Объем_фасовки, т"
0,ВНП,1779443,2022-06-11,0


In [44]:
orders

,КССС мат,Пункт отгрузки,"осталось произвести, тн",дата производства
0,1779443,ВНП,8,2022-06-11


In [712]:
graphic.loc[(graphic['Пункт отгрузки']=='ПНОС') & (graphic['Код КССС']==207465), 'Объем_фасовки, т']

Series([], Name: Объем_фасовки, т, dtype: int64)

In [713]:
graphic[(graphic['Пункт отгрузки']=="ПНОС") & (graphic['Код КССС']==20746)].empty

True

In [714]:
##делать колонку с тем, сколько не хватает еще бессмысленно, так как продукт может далее распределиться по другим заказам, 
#для которых хватает и тогда число не будет иметь значение

In [715]:
# проверить если не накопилась дважды или трижды